In [2]:
spark.stop()

In [ ]:
spark.sparkContext.stop()

In [ ]:
exit()

In [ ]:
!ls -la /lessons

In [ ]:
!pwd

In [ ]:
!hadoop fs -help

In [ ]:
!hadoop fs -ls /user/data/master/events

In [ ]:
!hadoop fs -head /user/master/data/geo/events/date=2022-05-30/part-00013-82f846d5-74a4-44e0-8fa9-de643f825932.c000.snappy.parquet

In [2]:
!hdfs dfs -rm -r /user/nickperegr/prj7/

Deleted /user/nickperegr/prj7


In [ ]:
!hadoop fs -cat /user/master/data/events/date=2022-05-31/part-00050-e1fe6a42-638b-4ad4-adc9-c7d0d312eef3.c000.json

In [ ]:
!hadoop fs -head /user/master/data/events/date=2022-06-08/part-00084-e1fe6a42-638b-4ad4-adc9-c7d0d312eef3.c000.json

In [ ]:
!hdfs dfs -ls /user/nickperegr/data/events/

In [10]:
!hdfs dfs -put -f geo.csv /user/nickperegr/data/cities

In [ ]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("My first session") \
        .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/31 02:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("My second session") \
        .getOrCreate() 

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Learning DataFrames") \
                    .getOrCreate()

schema = StructType([
    StructField("Name", StringType(), nullable=True),
    StructField("Age", LongType(), nullable=True),])

data = [('Max', 55),
        ('Yan', 53),
        ('Dmitry', 54),
        ('Ann', 25),]

df = spark.createDataFrame(data=data, schema=schema) 
df.printSchema()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("Learning DataFrames") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/date=2022-06-21/*.json")

events.show(10, False);

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("local")\
        .appName("Learning DataFrames") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/")

events.show(10);

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("Sorting channel Type") \
        .getOrCreate()

df = spark.read.parquet("/user/master/data/snapshots/channels/actual/*.parquet")

df.show(10);

df.write.format("parquet").mode("overwrite").partitionBy("channel_type").save('/user/nickperegr/analytics/test')

df1 = spark.read.parquet("/user/nickperegr/analytics/test/")

df1.select("channel_type").orderBy(desc("channel_type")).distinct().show()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Learning DataFrames") \
                    .getOrCreate()
# данные первого датафрейма 
book = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller',  174),
        ('The Catcher in the Rye', 'J. D. Salinger',  168),
        ('The Wind in the Willows', 'Kenneth Grahame',  259),
        ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
        ('Bad Girls', 'Jacqueline Wilson',  299)
]
# данные второго датафрейма
library = [
        ( 322, "1"),
        ( 250, "2" ),
        (400, "2"),
        (159, "1"),
        (382, "2"),
        (322, "1")
]
# названия атрибутов
schema = StructType([
    StructField("title", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("book_id", LongType(), nullable=True),])

columns_library = ['book_id', 'Library_id']
# создаём датафреймы
df = spark.createDataFrame(data=book, schema=schema)
df_library  = spark.createDataFrame(data=library, schema=columns_library )

df_joined = df.join(df_library, ["book_id"], how = "left_anti").select(df.title)

df_joined.show()

df_joined_1 = df.join(df_library.filter("Library_id == '1'"), ["book_id"], how = "left_anti")
df_joined_2 = df.join(df_library.filter("Library_id == '2'"), ["book_id"], how = "left_anti")

df_joined_1.union(df_joined_2).show()

df_joined_3 = df.join(df_library, ["book_id"], how = "left").collect()

df_joined_3


In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Learning DataFrames") \
                    .getOrCreate()
# данные первого датафрейма 
book_1 = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller',  174),
        ('The Catcher in the Rye', 'J. D. Salinger',  168),
        ('The Wind in the Willows', 'Kenneth Grahame',  259),
        ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
        ('Bad Girls', 'Jacqueline Wilson',  299)
]
# данные второго датафрейма
book_2 = [
        ('Black Beauty',657 ,'Anna Sewell'),
        ('Artemis Fowl',558,'Eoin Colfer'),
        ('The Magic Faraway Tree', 567,'Enid Blyton'),
        ('The Witches', 567,'Roald Dahl'),
        ('Frankenstein',567 ,'Mary Shelley'),
        ('The Little Prince',557 ,'Antoine de Saint-Exupéry'),
        ('The Truth', 576 ,'Terry Pratchett')
]
# названия атрибутов
columns_1= ['title', 'author', 'book_id']
columns_2 = ['title', 'book_id', 'author']
# создаём датафреймы
df_1 = spark.createDataFrame(data=book_1 , schema=columns_1)
df_2  = spark.createDataFrame(data=book_2 , schema=columns_2)
# напишите ваш код ниже
df = df_1.unionByName(df_2)

df.show(truncate=0)
df.show(df.count(),False)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Cache") \
                    .getOrCreate()
# данные первого датафрейма
book = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller',  174),
        ('The Catcher in the Rye', 'J. D. Salinger',  168),
        ('The Wind in the Willows', 'Kenneth Grahame',  259),
        ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
        ('Bad Girls', 'Jacqueline Wilson',  299)
]
# данные второго датафрейма
library = [
        ( 322, "1"),
        ( 250, "2" ),
        (400, "2"),
        (159, "1"),
        (382, "2"),
        (322, "1")
]
# названия атрибутов
schema = StructType([
    StructField("title", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("book_id", LongType(), nullable=True),])

columns_library = ['book_id', 'Library_id']
# создаём датафреймы
df = spark.createDataFrame(data=book, schema=schema)
df_library  = spark.createDataFrame(data=library, schema=columns_library )

df_joined = df.join(df_library, ["book_id"], how = "left_anti").select(df.title)

df_joined.cache()

df_joined.show()

df_joined.explain()

In [ ]:
import pyspark
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *


sc = SparkContext.getOrCreate(SparkConf())
sc.setCheckpointDir(dirName="/user/nickperegr/content")

spark = SparkSession.builder \
                    .master("local") \
                    .appName("CheckPoint") \
                    .getOrCreate()
# данные первого датафрейма
book = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller',  174),
        ('The Catcher in the Rye', 'J. D. Salinger',  168),
        ('The Wind in the Willows', 'Kenneth Grahame',  259),
        ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
        ('Bad Girls', 'Jacqueline Wilson',  299)
]
# данные второго датафрейма
library = [
        ( 322, "1"),
        ( 250, "2" ),
        (400, "2"),
        (159, "1"),
        (382, "2"),
        (322, "1")
]
# названия атрибутов
schema = StructType([
    StructField("title", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("book_id", LongType(), nullable=True),])

columns_library = ['book_id', 'Library_id']
# создаём датафреймы
df = spark.createDataFrame(data=book, schema=schema)
df_library  = spark.createDataFrame(data=library, schema=columns_library )

df_joined = df.join(df_library, ["book_id"], how = "left_anti").select(df.title)

df_joined.checkpoint()

df_joined.explain()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("func") \
        .getOrCreate()

events_curr_day = spark.read.json("/user/master/data/events/")
#events= spark.read.json("/user/master/data/events/date=2022-06-21/*.json")
#events.show(10)
#events_curr_day = events.withColumn("date", F.current_date())
events_curr_day.show(10)

events1 = events_curr_day.withColumn("current_hour", F.hour(events_curr_day.datetime))
events2 = events1.withColumn("current_minute", F.minute(events1.datetime))
events3 = events2.withColumn("current_second", F.second(events2.datetime))

events4 = events3.orderBy(F.desc("datetime"))

events4.show(10)


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("func") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/")
events.filter(F.col('event.message_to').isNotNull()).count()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("func") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/")
events_type = events.select(events.event_type).distinct()
events_type.show(events_type.count(), False)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("reaction") \
        .getOrCreate()

events = spark.read.json("/user/master/data/events/")

events_f = events.filter(F.col("event_type")=="reaction") \
       .filter(F.col("event.datetime").isNotNull()) \
       .groupBy(F.to_date(F.col("event.datetime"))).count()

events_f.orderBy(F.desc('count')).show() 

In [ ]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.window import Window 
import pyspark.sql.functions as F

spark = SparkSession.builder \
                    .master("yarn") \
                    .appName("Window") \
                    .getOrCreate()
# данные  датафрейма 
data = [('2021-01-06', 3744, 63, 322),
        ('2021-01-04', 2434, 21, 382),
        ('2021-01-04', 2434, 32, 159),
        ('2021-01-05', 3744, 32, 159),
        ('2021-01-06', 4342, 32, 159),
        ('2021-01-05', 4342, 12, 259),
        ('2021-01-06', 5677, 12, 259),
        ('2021-01-04', 5677, 23, 499)
]
# названия атрибутов
columns = ['dt', 'user_id', 'product_id', 'purchase_amount']
# создаём датафрейм
df = spark.createDataFrame(data=data, schema=columns) 

#window = Window().partitionBy(['purchase_amount']).orderBy(F.asc('purchase_amount'))
window = Window().orderBy(F.asc('purchase_amount'), F.asc('user_id'))

df_window = df.withColumn("row_number", F.row_number().over(window))

df_window.select("dt", "user_id", "purchase_amount", "row_number") \
.show()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window  import Window
import pyspark.sql.functions as F

spark = SparkSession.builder \
                    .master("yarn") \
                    .appName("lag") \
                    .getOrCreate()

events = spark.read.json("/user/master/data/events/")

window = Window().partitionBy('event.message_from').orderBy(F.col('date'))

dfWithLag = events.withColumn("lag_7",F.lag("event.message_to", 7).over(window))

dfWithLag.select("event.message_from", "date", "lag_7") \
.filter(dfWithLag.lag_7.isNotNull()) \
.orderBy(F.desc("date")).show(10, False)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window 
import pyspark.sql.functions as F

spark = SparkSession.builder \
                    .master("local") \
                    .config("spark.driver.allowMultipleContexts", "true") \
                    .appName("min-max") \
                    .getOrCreate()

data = [('2021-01-06', 3744, 63, 322),
        ('2021-01-04', 2434, 21, 382),
        ('2021-01-04', 2434, 32, 159),
        ('2021-01-05', 3744, 32, 159),
        ('2021-01-06', 4342, 32, 159),
        ('2021-01-05', 4342, 12, 259),
        ('2021-01-06', 5677, 12, 259),
        ('2021-01-04', 5677, 23, 499)
]

columns = ['dt', 'user_id', 'product_id', 'purchase_amount']

df = spark.createDataFrame(data=data, schema=columns)

window = Window().partitionBy(F.col('user_id')).orderBy(F.col('user_id'))

df_window = df.withColumn("min", F.min(F.col('purchase_amount')).over(window)) \
              .withColumn("max", F.max(F.col('purchase_amount')).over(window))

df_window.select('dt', 'user_id', 'max', 'min').distinct().show()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .appName("ODS") \
        .getOrCreate()

df = spark.read.json("/user/master/data/events/")

#df1 = df.filter(df.date <= F.to_date(F.lit("2022-05-31"))) 

df1.write.format("parquet") \
    .mode("overwrite") \
    .partitionBy("date", "event_type") \
    .save("/user/nickperegr/data/events/")

df2 = spark.read.parquet("/user/nickperegr/data/events/")

df2.select("event", "date", "event_type") \
    .filter(F.col("event.datetime").isNotNull()) \
    .orderBy(F.col("event.datetime").desc()).show(10)

df2.select("event", "date", "event_type") \
    .filter(F.col("date") <= F.to_date(F.lit("2022-05-31"))) \
    .orderBy(F.col("event.datetime").desc()).show(10)

In [ ]:
!spark-submit --master yarn --deploy-mode cluster partition.py 2022-05-31 /user/master/data/events /user/nickperegr/data/events

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("test") \
        .getOrCreate()

df = spark.read.parquet("/user/master/data/snapshots/tags_verified/actual")

df.printSchema()

df.show(10);


In [ ]:
import sys
from datetime import datetime, timedelta

def input_paths(date, depth):

# Вычитаем дни из даты
    end = datetime.strptime(date, "%Y-%m-%d")
    start = end - timedelta(days=depth-1)
    
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days+1)]
    
    paths = list()
    
    for date in date_generated:
        tag_date = date.strftime("%Y-%m-%d")
        paths.append(f"/user/nickperegr/data/events/date={tag_date}/event_type=message")
    
    return paths

In [ ]:
paths = input_paths('2022-05-31', 7)
print(paths)

In [ ]:
import pyspark
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

def input_paths(date, depth):

# Вычитаем дни из даты
    end = datetime.strptime(date, "%Y-%m-%d")
    start = end - timedelta(days=depth-1)
    
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days+1)]
    
    paths = list()
    
    for date in date_generated:
        tag_date = date.strftime("%Y-%m-%d")
        paths.append(f"/user/nickperegr/data/events/date={tag_date}/event_type=message")
    
    return paths

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("test") \
        .getOrCreate()


paths = input_paths('2022-05-31', 84)

df = spark.read.parquet(*paths)

#если в схеме есть массив, то нужно использовать explode
df_comp = df.filter( F.col('event.message_channel_to').isNotNull())\
            .selectExpr('event.message_from as user', 'explode(event.tags) as tag')\
            .groupBy('tag')\
            .agg(F.countDistinct('user').alias('suggested_count'))\
            .where('suggested_count >= 2000')

df_tags_verified = spark.read.parquet("/user/master/data/snapshots/tags_verified/actual")

df_joined = df_comp.join(df_tags_verified, ["tag"], how = "left_anti")

df_joined.write\
    .mode("overwrite")\
    .format("parquet")\
    .save("/user/nickperegr/data/analytics/candidates_d84_pyspark")

df = spark.read.parquet('/user/nickperegr/data/analytics/candidates_d84_pyspark')

df.describe().show()

In [ ]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

def input_paths(date, depth):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"/user/nickperegr/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}/event_type=message" for x in range(depth)]

paths = input_paths('2022-05-31', 7)

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("test") \
        .getOrCreate()

messages = spark.read.parquet(*paths)

all_tags = messages.where("event.message_channel_to is not null")\
.selectExpr(["event.message_from as user", "explode(event.tags) as tag"])\
.groupBy("tag").agg(F.expr("count(distinct user) as suggested_count"))\
.where("suggested_count >= 100")

#all_tags.orderBy(F.desc('suggested_count')).show()

verified_tags = spark.read.parquet("/user/master/data/snapshots/tags_verified/actual")
candidates = all_tags.join(verified_tags, "tag", "left_anti")

candidates.write.mode("overwrite").parquet('/user/nickperegr/data/analytics/candidates_d7_pyspark')

In [ ]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("test") \
        .getOrCreate()

df = spark.read.parquet('/user/nickperegr/data/analytics/candidates_d84_pyspark')

df.describe().show()

In [ ]:
from datetime import datetime, timedelta

def input_paths(date, depth, base_input_path):

# Вычитаем дни из даты
    end = datetime.strptime(date, "%Y-%m-%d")
    start = end - timedelta(days=depth-1)
    
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days+1)]
    
    paths = list()
    
    for date in date_generated:
        tag_date = date.strftime("%Y-%m-%d")
        paths.append(f"{base_input_path}/date={tag_date}/event_type=message")
    
    return paths

In [ ]:
paths = input_paths('2022-06-04', 5, '/user/nickperegr/data/events')
print(paths)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window 
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("tag_tops") \
        .getOrCreate()

messages = spark.read.parquet(*paths)

messages.printSchema()

all_tags = messages.where("event.message_channel_to is not null")\
    .selectExpr(["event.message_id", "event.message_from as user_id", "explode(event.tags) as tag"])\
    .groupBy("user_id", "tag")\
    .agg(F.count("tag").cast(LongType()).alias("suggested_count"))\

window = Window.partitionBy("user_id").orderBy(F.desc("tag"), F.desc("suggested_count"))

df_pivoted = all_tags.withColumn("rn", F.row_number().over(window))\
    .filter(F.col("rn").cast(LongType()) <= 3)\
    .drop("suggested_count")\
    .groupBy("user_id").pivot("rn").agg(F.first("tag"))

result = df_pivoted.withColumnRenamed("1","tag_top_1")\
           .withColumnRenamed("2","tag_top_2")\
           .withColumnRenamed("3","tag_top_3")
    
result.show(20, False)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window 
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("tag_tops") \
        .getOrCreate()

result = spark.read \
        .parquet(*paths) \
        .where("event.message_channel_to is not null") \
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag")) \
        .groupBy("user_id", "tag") \
        .agg(F.count("*").alias("tag_count")) \
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id") \
                                                .orderBy(F.desc("tag"), F.desc("tag_count")))) \
        .where("rank <= 3") \
        .groupBy("user_id") \
        .pivot("rank", [1, 2, 3]) \
        .agg(F.first("tag")) \
        .withColumnRenamed("1", "tag_top_1") \
        .withColumnRenamed("2", "tag_top_2") \
        .withColumnRenamed("3", "tag_top_3")

result.show(20, False)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window 
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("tag_tops") \
        .getOrCreate()

result = spark.read \
        .parquet(*paths) \
        .where("event.message_channel_to is not null") \
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag"))\
        .groupBy("user_id", "tag") \
        .agg(F.count("*").alias("tag_count"))\
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id") \
        .orderBy(F.desc("tag"), F.desc("tag_count"))))

print(result.count())


result1 = spark.read \
        .parquet(*paths) \
        .where("event.message_channel_to is not null") \
        .select(F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag"))\
        .groupBy("user_id", "tag") \
        .agg(F.count("*").alias("tag_count"))\
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id") \
        .orderBy(F.desc("tag"), F.desc("tag_count"))))

print(result1.count())


In [ ]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window


def input_event_paths(date, depth):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"/user/nickperegr/data/events/date={(dt - datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in
            range(depth)]


def reaction_tag_tops(date, depth, spark):
    reaction_paths = input_event_paths(date, depth)
    reactions = spark.read \
        .option("basePath", "/user/nickperegr/data/events") \
        .parquet(*reaction_paths) \
        .where("event_type='reaction'")

    all_message_tags = spark.read.parquet("/user/nickperegr/data/events") \
        .where("event_type='message' and event.message_channel_to is not null") \
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag")
                )

    reaction_tags = reactions \
        .select(F.col("event.reaction_from").alias("user_id"),
                F.col("event.message_id").alias("message_id"),
                F.col("event.reaction_type").alias("reaction_type")
                ).join(all_message_tags.select("message_id", "tag"), "message_id")

    reaction_tops = reaction_tags \
        .groupBy("user_id", "tag", "reaction_type") \
        .agg(F.count("*").alias("tag_count")) \
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id", "reaction_type") \
                                                .orderBy(F.desc("tag"), F.desc("tag_count")))) \
        .where("rank <= 3") \
        .groupBy("user_id", "reaction_type") \
        .pivot("rank", [1, 2, 3]) \
        .agg(F.first("tag")) \
        .cache()

    like_tops = reaction_tops \
        .where("reaction_type = 'like'") \
        .drop("reaction_type") \
        .withColumnRenamed("1", "like_tag_top_1") \
        .withColumnRenamed("2", "like_tag_top_2") \
        .withColumnRenamed("3", "like_tag_top_3")

    dislike_tops = reaction_tops \
        .where("reaction_type = 'dislike'") \
        .drop("reaction_type") \
        .withColumnRenamed("1", "dislike_tag_top_1") \
        .withColumnRenamed("2", "dislike_tag_top_2") \
        .withColumnRenamed("3", "dislike_tag_top_3")

    result = like_tops \
        .join(dislike_tops, "user_id", "full_outer")
    print("here 1 ")
    result.write.mode("overwrite").parquet(f"/user/nickperegr/data/tmp/reaction_tag_tops_{date[5:7]}_{date[8:10]}_{depth}")
    print(f"save /user/nickperegr/data/tmp/reaction_tag_tops_{date[5:7]}_{date[8:10]}_{depth}")
    return None

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("reaction_tag_tops") \
        .getOrCreate()

depths=[5,5,1]
dates=['2022-05-04', '2022-04-04', '2022-04-04']
s = zip(dates,depths)
for date,depth in s:
    reaction_tag_tops(date, depth, spark)


In [ ]:
message = "2022-04-04"
print (message[-11111:2]+'_'+message[8:1111])

In [ ]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window

def input_event_paths(date, depth):
        dt = datetime.datetime.strptime(date, '%Y-%m-%d')
        return [f"/user/nickperegr/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in range(depth)]
def tag_tops(date, depth, spark):
	
    message_paths = input_event_paths(date, depth)
	
    result = spark.read\
    .option("basePath", "/user/nickperegr/data/events")\
    .parquet(*message_paths)\
    .where("event_type = 'message'")\
		.where("event.message_channel_to is not null")\
		.select(F.col("event.message_id").alias("message_id"),
		        F.col("event.message_from").alias("user_id"),
		        F.explode(F.col("event.tags")).alias("tag"))\
		.groupBy("user_id", "tag")\
		.agg(F.count("*").alias("tag_count"))\
		.withColumn("rank", F.row_number().over(Window.partitionBy("user_id")\
        .orderBy(F.desc("tag"), F.desc("tag_count"))))\
		.where("rank <= 3")\
		.groupBy("user_id")\
		.pivot("rank", [1, 2, 3])\
		.agg(F.first("tag"))\
		.withColumnRenamed("1", "tag_top_1")\
		.withColumnRenamed("2", "tag_top_2")\
		.withColumnRenamed("3", "tag_top_3")
	
    return result

def reaction_tag_tops(date, depth, spark):
	
    reaction_paths = input_event_paths(date, depth)
	
    reactions = spark.read\
    .option("basePath", "/user/nickperegr/data/events")\
    .parquet(*reaction_paths)\
    .where("event_type = 'reaction'")
	
    all_message_tags = spark.read.parquet(f"{events_base_path}")\
		.where("event_type='message' and event.message_channel_to is not null")\
		.select(F.col("event.message_id").alias("message_id"),
		        F.col("event.message_from").alias("user_id"),
		        F.explode(F.col("event.tags")).alias("tag")
		)
	
    reaction_tags = reactions\
		.select(F.col("event.reaction_from").alias("user_id"),
		        F.col("event.message_id").alias("message_id"),
		        F.col("event.reaction_type").alias("reaction_type")
		).join(all_message_tags.select("message_id", "tag"), "message_id")
	
    reaction_tops = reaction_tags\
		.groupBy("user_id", "tag", "reaction_type")\
		.agg(F.count("*").alias("tag_count"))\
		.withColumn("rank", F.row_number().over(Window.partitionBy("user_id", "reaction_type")\
        .orderBy(F.desc("tag"), F.desc("tag_count"))))\
		.where("rank <= 3")\
		.groupBy("user_id", "reaction_type")\
		.pivot("rank", [1, 2, 3])\
		.agg(F.first("tag"))\
		.cache()
	
    like_tops = reaction_tops\
		.where("reaction_type = 'like'")\
		.drop("reaction_type")\
		.withColumnRenamed("1", "like_tag_top_1")\
		.withColumnRenamed("2", "like_tag_top_2")\
		.withColumnRenamed("3", "like_tag_top_3")
	
    dislike_tops = reaction_tops\
		.where("reaction_type = 'dislike'")\
		.drop("reaction_type")\
		.withColumnRenamed("1", "dislike_tag_top_1")\
		.withColumnRenamed("2", "dislike_tag_top_2")\
		.withColumnRenamed("3", "dislike_tag_top_3")
	
    result = like_tops\
		.join(dislike_tops, "user_id", "full_outer")

    return result

def calculate_user_interests(date, depth, filename, spark):

    post_tops = tag_tops(date, depth, spark)
    
    reaction_tops = reaction_tag_tops(date, depth, spark)
    
    result = post_tops.join(reaction_tops, "user_id", "full_outer")

    result.write.mode("overwrite").parquet(f"/user/nickperegr/data/analytics/{filename}")

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("user_interests") \
        .getOrCreate()

depths=[5,5,1]
dates=['2022-04-04', '2022-05-04', '2022-04-04']
filenames=['user_interests_04_04_5', 'user_interests_04_05_5', 'user_interests_04_04_1']
s = zip(dates,depths,filenames)
for date,depth,filename in s:
    calculate_user_interests(date, depth, filename, spark)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("user_interests") \
        .getOrCreate()

depths=[7,28]
dates=['2022-05-31', '2022-05-31']
filenames=['user_interests_05_31_7', 'user_interests_05_31_28']
s = zip(dates,depths,filenames)
for date,depth,filename in s:
    calculate_user_interests(date, depth, filename, spark)

In [ ]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window


def input_event_paths(date, depth, events_base_path):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"{events_base_path}/date={(dt - datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in
            range(depth)]

def connection_interests(date, days_count, events_base_path, spark):
	
    message_paths = input_event_paths(f"{date}", int(days_count), f"{events_base_path}")

    all_message_from = spark.read.parquet(f"{events_base_path}") \
        .where("event_type='message' and event.message_channel_to is not null") \
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag")
                )

    all_message_from.printSchema()
    
    all_message_from.show(20)    
    
    all_message_to = spark.read.parquet(f"{events_base_path}") \
        .where("event_type='message' and event.message_channel_to is not null") \
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_to").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag")
                )

    all_message_to.printSchema()
    
    all_message_to.show(20)


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("connection_interests") \
        .getOrCreate()

depths=[5]
dates=['2022-05-25']
b_paths=['/user/nickperegr/data/events']
s = zip(dates,depths,b_paths)
for date,depth,b_path in s:
    connection_interests(date, depth, b_path, spark)

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .appName("test5") \
        .getOrCreate()

#spark.read.parquet("/user/master/data/snapshots/users/actual").where("age > 18").groupBy("id").agg(F.avg("age").alias("avgage")).repartition(3).where("avgage < 50").distinct().show()
#spark.read.parquet("/user/master/data/snapshots/users/actual").where("age > 18").groupBy("id").agg(F.avg("age")).collect()
#spark.read.parquet("/user/master/data/snapshots/users/actual").where("age > 18").groupBy("id").agg(F.avg("age").alias("avgage")).show()

df1 = spark.read.parquet("/user/examples/users_small")

F.broadcast(df1)

df2 = spark.read.parquet("/user/nickperegr/data/events")

df2.printSchema()

cond = [df1.id == df2.event.message_from]

df = df1.join(df2, cond, "inner")

df.explain(True)

df.show()


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
        .appName("test6") \
        .getOrCreate()

#spark.read.parquet("/user/master/data/snapshots/users/actual").where("age > 18").groupBy("id").agg(F.avg("age").alias("avgage")).repartition(3).where("avgage < 50").distinct().show()
#spark.read.parquet("/user/master/data/snapshots/users/actual").where("age > 18").groupBy("id").agg(F.avg("age")).collect()
#spark.read.parquet("/user/master/data/snapshots/users/actual").where("age > 18").groupBy("id").agg(F.avg("age").alias("avgage")).show()

df1 = spark.read.parquet("/user/examples/users_small")

F.broadcast(df1)

df2 = spark.read.parquet("/user/nickperegr/data/events")

df2.printSchema()

cond = [df1.id == df2.event.message_from]

df = df1.join(df2, cond, "inner")

df.explain(True)

df.show()


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
        .appName("test6") \
        .getOrCreate()

user_bd = spark.read.parquet("/user/examples/user_birthdays")

events = spark.read.parquet("/user/nickperegr/data/events/")

res = user_bd.join(events.select("event.*"), F.col("id") == F.col("message_from"), "left_outer")\
.groupBy("id", "birthday").agg(F.count(F.when(F.expr("message like '%birthday%'"), 
F.lit(1))).alias("count"))

#res.explain(True)

res.write.mode("overwrite").parquet(f"/user/nickperegr/data/analytics/user_birthdays")


In [ ]:
events.printSchema()

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .master("yarn")\
        .config("spark.executor.memory", "1g") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.memory", "1g") \
        .config("spark.driver.cores",2) \
        .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
        .appName("test6") \
        .getOrCreate()

user_bd = spark.read.parquet("/user/examples/user_birthdays")

events = spark.read.parquet("/user/nickperegr/data/events/")

res = user_bd\
.select("id")\
.distinct()\
.join(events.select("event.*"), F.col("id") == F.col("message_from"), "inner")\
.groupBy("id").agg(F.count(F.when(F.expr("message like '%birthday%'"), F.lit(1))).alias("count"))\
.join(user_bd, "id", "right_outer") 

res.explain(True)

res.write.mode("overwrite").parquet(f"/user/nickperegr/data/analytics/user_birthdays")

In [ ]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Learning DataFrames") \
        .getOrCreate()

events = spark.read.parquet("/user/master/data/geo/events")

events.printSchema()

events.select("event_type", "event.subscription_user", "event.datetime", "event.message_ts", "lat", "lon")\
.where("event_type == 'subscription' and subscription_user is not null and subscription_user = 165829")\
.show(20, False)


In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("local")\
        .appName("Learning DataFrames") \
        .getOrCreate()

cities = spark.read.format("csv").option("delimiter", ";").option("header", "true").load("/user/nickperegr/data/cities")

cities.printSchema()

cities.show(20)

cities.write.mode("overwrite").partitionBy("city").parquet("/user/nickperegr/data/cities_part")

22/12/03 17:53:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)

+---+----------+--------+--------+
| id|      city|     lat|     lng|
+---+----------+--------+--------+
|  1|    Sydney| -33,865|151,2094|
|  2| Melbourne|-37,8136|144,9631|
|  3|  Brisbane|-27,4678|153,0281|
|  4|     Perth|-31,9522|115,8589|
|  5|  Adelaide|-34,9289|138,6011|
|  6|Gold Coast|-28,0167|   153,4|
|  7|Cranbourne|-38,0996|145,2834|
|  8|  Canberra|-35,2931|149,1269|
|  9| Newcastle|-32,9167|  151,75|
| 10|Wollongong|-34,4331|150,8831|
| 11|   Geelong|  -38,15|  144,35|
| 12|    Hobart|-42,8806| 147,325|
| 13|Townsville|-19,2564|146,8183|
| 14|   Ipswich|-27,6167|152,7667|
| 15|    Cairns|-16,9303|145,7703|
| 16| Toowoomba|-27,5667|  151,95|
| 17|    Darwin|-12,4381|130,8411|
| 18|  Ballarat|  -37,55|  143,85|
| 19|   Bendigo|  -36,75|144,2667|
| 20|Launceston|-41,4419| 147,145|
+---+----------+--------+--------+
only showing t

In [117]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.window import Window
import sys
import datetime
from datetime import datetime
from datetime import timedelta
from math import radians, cos, sin, asin, sqrt

#Вспомогательные функции - определение временных зон, загрузка справочников городов, получение локального времени
def create_tz_dataset(spark):

    schema = StructType([
    StructField("city", StringType(), nullable=False),
    StructField("tz_city", StringType(), nullable=False),])

    tz = [
            ('Sydney', 'Sydney'),
            ('Melbourne', 'Melbourne'),
            ('Brisbane', 'Brisbane'),
            ('Perth', 'Perth'),
            ('Adelaide', 'Adelaide'),
            ('Gold Coast', 'Brisbane'),
            ('Cranbourne', 'Melbourne'),
            ('Canberra', 'Canberra'),
            ('Newcastle', 'Sydney'),
            ('Wollongong', 'Sydney'),
            ('Geelong', 'Melbourne'),
            ('Hobart', 'Hobart'),
            ('Townsville', 'Brisbane'),
            ('Ipswich', 'Brisbane'),
            ('Cairns', 'Brisbane'),
            ('Toowoomba', 'Brisbane'),
            ('Darwin', 'Darwin'),
            ('Ballarat', 'Melbourne'),
            ('Bendigo', 'Melbourne'),
            ('Launceston', 'Hobart'),
            ('Mackay', 'Brisbane'),
            ('Rockhampton', 'Brisbane'),
            ('Maitland', 'Sydney' ),
            ('Bunbury', 'Perth')
    ]

    tz_df = spark.createDataFrame(data=tz, schema=schema)

    return tz_df

def load_cities(spark):
    cities = spark.read.format("csv").option("delimiter", ";").option("header", "true").load("/user/nickperegr/data/cities") \
                  .withColumn("lat1", F.regexp_replace(F.col("lat"), ',', '.').cast("float"))\
                  .withColumn("lon1", F.regexp_replace(F.col("lng"), ',', '.').cast("float"))\
                  .drop("lat", "lng")

    return cities

def load_local_time(df, left_join_col, time_col, spark):

    tz = create_tz_dataset(spark)
    
    df_tz = df.join(tz, df[left_join_col] == tz["city"], "inner").drop(tz.city).withColumnRenamed("tz_city", "city")
    
    local_time = df_tz.withColumn("ts_timestamp",F.col(time_col).cast("Timestamp"))\
    .withColumn("timezone",F.concat(F.lit("Australia/"),F.col("city")))\
    .withColumn("local_time",F.from_utc_timestamp(F.col("ts_timestamp"),F.col("timezone")))\
    .drop("timezone", "ts_timestamp")    
    
    return local_time

#Витрина пользователей
def load_geo_events_cities(date, days_delta, events_base_path, output_base_path, spark):

    geo_df = load_cities(spark)

    F.broadcast(geo_df)

    end_date = datetime.strptime(date, '%Y-%m-%d')
    start_date = end_date - timedelta(days=int(days_delta))

    geo_events_base = spark.read.parquet(events_base_path)\
                   .filter(F.col("date").between(start_date, end_date)) 

    geo_events = geo_events_base\
    .withColumn("user_id",F.when(F.col("event_type")=="message",F.col("event.message_from"))\
                    .when(F.col("event_type")=="reaction",F.col("event.reaction_from"))\
                    .when(F.col("event_type")=="subscription", F.coalesce(F.col("event.subscription_user"),F.col("event.user"))))\
    .withColumn("ts",F.when(F.col("event_type")=="message",F.coalesce(F.col("event.message_ts"),F.col("event.datetime")))\
                    .when(F.col("event_type")=="reaction",F.col("event.datetime"))\
                    .when(F.col("event_type")=="subscription",F.col("event.datetime")))\
    .filter(F.col("user_id").isNotNull())

    window = Window().partitionBy("user_id", "ts").orderBy("distance")

    geo_events_cities=geo_events.crossJoin(geo_df)\
        .where("lat is not null and lon is not null")\
        .withColumn("ts", F.to_timestamp(F.substring(F.col("ts"),1,19),"yyyy-MM-dd HH:mm:ss"))\
        .withColumn("distance", 2*6378*F.sqrt(F.pow(F.sin((F.col("lat1")-F.col("lat"))/2),2) + F.cos("lat")*F.cos("lat1")*F.pow(F.sin((F.col("lon1")-F.col("lon"))/2),2)))\
        .withColumn("rn", F.row_number().over(window))\
        .where("rn=1")\
        .drop("lat1", "lon1", "rn", "id", "distance")\
        .persist()

    geo_events_cities.write.mode("overwrite").partitionBy("date").parquet(f"{output_base_path}/geo_events_cities")

    return geo_events_cities

def load_home_cities(df):

    window = Window().partitionBy("user_id", "city").orderBy("date") 

    window1 = Window().partitionBy("user_id").orderBy("date")
    
    window2 = Window().partitionBy("user_id").orderBy("ts") 

    home_cities = df\
      .select("user_id", "ts", "date", "city")\
      .withColumn("days_rank", F.dense_rank().over(window))\
      .withColumn("start_dt",F.min(F.col("date")).over(window))\
      .withColumn("end_dt",F.max(F.col("date")).over(window))\
      .withColumn("days", F.datediff(F.col("end_dt"),F.col("start_dt"))+1)\
      .withColumn("prev_city", F.lag("city", 1, "UnknownCity").over(window2))\
      .withColumn("home_city",F.when((F.col("days")>=27) & (F.col("days")==F.col("days_rank")),F.col("city")))\
      .withColumn("home_city", F.last("home_city", ignorenulls=True).over(window1))\
      .filter(F.col("city") != F.col("prev_city"))\
      .withColumn("travel_count",F.size(F.collect_list("city").over(window2)))\
      .withColumn("travel_array", F.collect_list("city").over(window2))\
      .withColumn("rn",F.row_number().over(Window.partitionBy("user_id").orderBy(F.col("ts").desc())))\
      .filter(F.col("rn") == 1)\
      .select("user_id", "home_city", "travel_count", "travel_array")

    return home_cities

def load_geo_actual_cities(df):

    window = Window().partitionBy("user_id").orderBy(F.desc("ts"))

    geo_events_actual_cities = df\
        .withColumn("rn", F.row_number().over(window))\
        .where("rn == 1")\
        .withColumnRenamed("city", "act_city")\
        .withColumnRenamed("ts", "local_time")\
        .select("user_id", "local_time", "act_city")

    return geo_events_actual_cities

def load_geo_dm(date, days_count, events_base_path, output_base_path, output_mart_path, spark):
    geo_events_cities=load_geo_events_cities(date, days_count, events_base_path, output_base_path, spark)
    home_cities = load_home_cities(geo_events_cities)
    geo_events_actual_cities = load_geo_actual_cities(geo_events_cities)
    local_time_cities = load_local_time(geo_events_actual_cities, "act_city", "local_time", spark)

    cond = [home_cities.user_id == local_time_cities.user_id]
    user_geo = home_cities.join(local_time_cities, cond, "left")\
                             .drop(local_time_cities.user_id)\
                             .select("user_id", "act_city", "home_city", "travel_count", "travel_array", "local_time")

    user_geo.write.mode("overwrite").parquet(f"{output_mart_path}/user_geo")

In [118]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Project7") \
        .getOrCreate()

depths=[5]
dates=['2022-05-01']
b_paths=['/user/master/data/geo/events/']
o_paths=['/user/nickperegr/prj7/prod/data/events/']
m_paths=['/user/nickperegr/prj7/prod/data/analytics/']

#s = zip(dates,depths,b_paths, o_paths, m_paths)
#for date,depth,b_path, o_path, m_path in s:
    #geo_events_cities=load_geo_events_cities(date, depth, b_path, o_path, spark)
    #load_dm(date, depth, b_path, o_path, m_path, spark)
#geo_events_cities = spark.read.parquet("/user/nickperegr/prj7/prod/data/events/")    
#home_cities = load_home_cities(geo_events_cities)    
#geo_events_actual_cities = load_geo_actual_cities(geo_events_cities)
local_time_cities = load_local_time(geo_events_actual_cities, "act_city", "local_time", spark) 

In [ ]:
geo_events_cities.printSchema()

In [ ]:
geo_events_cities.show(100)

In [56]:
geo_events_cities.groupBy("user_id").count()\
  .filter(F.col("count") >= 2)\
  .show()

+-------+-----+
|user_id|count|
+-------+-----+
| 106506|    7|
| 112480|    5|
| 127075|    4|
| 157380|    6|
| 169689|    6|
|  18556|   13|
|  56315|    3|
|  57085|    5|
|  65348|   11|
|  70962|   10|
|  80422|   12|
|   8304|    5|
| 115640|    8|
| 148140|    5|
| 168278|    4|
|   2110|    9|
|  21645|    3|
|  35771|    9|
|  39755|    5|
|  49960|    9|
+-------+-----+
only showing top 20 rows



In [58]:
geo_events_cities.drop("lat", "lon").where("user_id= 18556").orderBy("ts").show(30)

+--------------------+------------+----------+-------+-------------------+----------+------------------+
|               event|  event_type|      date|user_id|                 ts|      city|          distance|
+--------------------+------------+----------+-------+-------------------+----------+------------------+
|{null, null, null...|     message|2022-04-26|  18556|2021-04-26 03:33:01|Wollongong|1981.8815744285155|
|{null, null, 2022...|subscription|2022-04-26|  18556|2022-04-26 02:29:11| Newcastle| 497.9938087946334|
|{null, null, 2022...|subscription|2022-04-26|  18556|2022-04-26 06:35:52|  Adelaide|1325.6469846287732|
|{null, null, 2022...|subscription|2022-04-26|  18556|2022-04-26 11:31:36| Newcastle|1681.0194460714886|
|{null, null, 2022...|subscription|2022-04-26|  18556|2022-04-26 13:34:39| Newcastle| 253.7102883924431|
|{null, null, 2022...|subscription|2022-04-26|  18556|2022-04-26 21:50:30|  Adelaide|1438.8911331305167|
|{null, null, 2022...|    reaction|2022-04-27|  18556|2

In [108]:
home_cities = load_home_cities(geo_events_cities)

In [109]:
home_cities.where("user_id= 18556").orderBy("ts").show(100, False)

+-------+---------+------------+----------------------------------------------------------------------------------------------------------------------+
|user_id|home_city|travel_count|travel_array                                                                                                          |
+-------+---------+------------+----------------------------------------------------------------------------------------------------------------------+
|18556  |Newcastle|11          |[Wollongong, Newcastle, Adelaide, Newcastle, Adelaide, Bunbury, Newcastle, Launceston, Toowoomba, Brisbane, Melbourne]|
+-------+---------+------------+----------------------------------------------------------------------------------------------------------------------+



In [110]:
geo_events_actual_cities = load_geo_actual_cities(geo_events_cities)

In [111]:
geo_events_actual_cities.where("user_id= 18556").orderBy("rn").show(20)

+-------+-------------------+---------+
|user_id|         local_time| act_city|
+-------+-------------------+---------+
|  18556|2022-04-30 09:43:24|Melbourne|
+-------+-------------------+---------+



In [119]:
local_time_cities.where("user_id= 18556").show(10)

+-------+-------------------+---------+
|user_id|         local_time| act_city|
+-------+-------------------+---------+
|  18556|2022-04-30 19:43:24|Melbourne|
+-------+-------------------+---------+



In [120]:
cond = [home_cities.user_id == local_time_cities.user_id]
user_geo_events = home_cities.join(local_time_cities, cond, "left")\
                             .drop(local_time_cities.user_id)\
                             .select("user_id", "act_city", "home_city", "travel_count", "travel_array", "local_time")

In [121]:
user_geo_events.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- act_city: string (nullable = true)
 |-- home_city: string (nullable = true)
 |-- travel_count: integer (nullable = false)
 |-- travel_array: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- local_time: timestamp (nullable = true)



In [123]:
user_geo_events.where("user_id= 18556").show(10,False)

+-------+---------+---------+------------+----------------------------------------------------------------------------------------------------------------------+-------------------+
|user_id|act_city |home_city|travel_count|travel_array                                                                                                          |local_time         |
+-------+---------+---------+------------+----------------------------------------------------------------------------------------------------------------------+-------------------+
|18556  |Melbourne|Newcastle|11          |[Wollongong, Newcastle, Adelaide, Newcastle, Adelaide, Bunbury, Newcastle, Launceston, Toowoomba, Brisbane, Melbourne]|2022-04-30 19:43:24|
+-------+---------+---------+------------+----------------------------------------------------------------------------------------------------------------------+-------------------+



In [67]:
user_geo_events.show(10)

+-------+----------+---------+------------+--------------------+-------------------+
|user_id|  act_city|home_city|travel_count|        travel_array|         local_time|
+-------+----------+---------+------------+--------------------+-------------------+
|      0|   Bendigo|     null|           5|[Toowoomba, Canbe...|2022-05-30 21:57:17|
|      1|  Canberra|     null|           5|[Toowoomba, Canbe...|2022-05-30 17:38:52|
|     10| Melbourne|     null|           3|[Bunbury, Melbour...|2022-05-28 21:39:34|
|    100| Toowoomba|     null|           7|[Toowoomba, Sydne...|2022-05-31 05:19:23|
|   1000|Wollongong|     null|           4|[Wollongong, Rock...|2022-05-31 09:34:28|
|  10000|Launceston|     null|           4|[Canberra, Adelai...|2022-05-31 07:32:35|
| 100000| Toowoomba|     null|           3|[Toowoomba, Launc...|2022-05-26 11:44:53|
| 100001|    Mackay|     null|           9|[Perth, Adelaide,...|2022-05-30 19:51:35|
| 100002|    Sydney|     null|           4|[Sydney, Bunbury,...|2

In [32]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Project 7") \
        .getOrCreate()

events = spark.read.parquet("/user/nickperegr/prj7/prod/data/analytics/user_geo/")

events.printSchema()

#events.where("home_city is not null").show(20, False)
events.show(10)

root
 |-- user_id: string (nullable = true)
 |-- act_city: string (nullable = true)
 |-- home_city: string (nullable = true)
 |-- travel_count: integer (nullable = true)
 |-- travel_array: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- local_time: timestamp (nullable = true)



+-------+----------+---------+------------+--------------------+-------------------+
|user_id|  act_city|home_city|travel_count|        travel_array|         local_time|
+-------+----------+---------+------------+--------------------+-------------------+
|      0|  Adelaide|     null|           4|[Adelaide, Bendig...|2022-04-30 18:45:11|
|      0|  Adelaide|     null|           2|   [Bendigo, Mackay]|2022-04-30 18:45:11|
|      0|  Adelaide|     null|           2|   [Bendigo, Mackay]|2022-04-30 18:45:11|
|      0|  Adelaide|     null|           3|[Bendigo, Newcast...|2022-04-30 18:45:11|
|     10|    Sydney|     null|           3|[Launceston, Bend...|2022-05-02 08:56:34|
|     10|    Sydney|     null|           2|[Launceston, Town...|2022-05-02 08:56:34|
|     10|    Sydney|     null|           4|[Sydney, Launcest...|2022-05-02 08:56:34|
|     10|    Sydney|     null|           4|[Sydney, Launcest...|2022-05-02 08:56:34|
|     10|    Sydney|     null|           1|        [Townsville]|2

In [74]:
events.where("home_city is not null").show(20)

+-------+--------+---------+------------+------------+----------+
|user_id|act_city|home_city|travel_count|travel_array|local_time|
+-------+--------+---------+------------+------------+----------+
+-------+--------+---------+------------+------------+----------+



In [75]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

def load_month_week_stat(input_events_path, spark):
    
    window_reg = Window.partitionBy("user_id","event_type").orderBy("ts")
    
    geo_events = spark.read \
     .parquet(f"{input_events_path}")
    
    
    events = geo_events.select(F.col("city").alias("zone_id"),\
                                    "event_type", "user_id", "date", "ts")\
     .where("user_id= 100310")\
     .withColumn("message_number",F.row_number().over(window_reg)) \
     .withColumn("month",F.trunc(F.col("date"), "month"))\
     .withColumn("week",F.trunc(F.col("date"), "week"))\
     .drop("user_id", "date", "ts")
    
    events_stat_m = events\
     .groupBy("zone_id", "month")\
     .agg(
         F.sum(F.when(geo_events.event_type == "message",1).otherwise(0)).alias("month_message"),
         F.sum(F.when(geo_events.event_type == "reaction",1).otherwise(0)).alias("month_reaction"),
         F.sum(F.when(geo_events.event_type == "subscription",1).otherwise(0)).alias("month_subscription"),
         F.sum(F.when((geo_events.event_type == "message") & (F.col("message_number")==1 ),1).otherwise(0)).alias("month_user")
    )
    
    events_stat_w = events\
     .groupBy("zone_id", "month", "week")\
     .agg(
         F.sum(F.when(geo_events.event_type == "message",1).otherwise(0)).alias("week_message"),
         F.sum(F.when(geo_events.event_type == "reaction",1).otherwise(0)).alias("week_reaction"),
         F.sum(F.when(geo_events.event_type == "subscription",1).otherwise(0)).alias("week_subscription"),
         F.sum(F.when((geo_events.event_type == "message") & (F.col("message_number")==1 ),1).otherwise(0)).alias("week_user")
    )
    
    cond = [events_stat_m.zone_id == events_stat_w.zone_id, events_stat_m.month == events_stat_w.month]
    events_stat = events_stat_m.join(events_stat_w, cond, "left")\
                               .drop(events_stat_w.zone_id)\
                               .drop(events_stat_w.month)
    
    return events_stat

def load_stat_dm(input_events_path, output_mart_path, spark):
    
    events_stat = load_month_week_stat(input_events_path, spark)

    events_stat.write.mode("overwrite").parquet(f"{output_mart_path}")    

In [76]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Project7") \
        .getOrCreate()

i_paths=['/user/nickperegr/prj7/prod/data/events/geo_events_cities/']
m_paths=['/user/nickperegr/prj7/prod/data/analytics/events_stat']

s = zip(i_paths, m_paths)
for i_path, m_path in s:
    #events_stat = load_month_week_stat(i_path, spark)
    load_stat_dm(i_path, m_path, spark)

22/11/30 16:27:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
events_stat.printSchema()


In [ ]:
events_stat.show()

In [ ]:
events_stat.show()

In [ ]:
events_stat.show()

In [78]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Project 7") \
        .getOrCreate()

events = spark.read.parquet("/user/nickperegr/prj7/prod/data/analytics/events_stat/")

events.printSchema()

#events.where("home_city is not null").show(20, False)
events.show(20)

22/11/30 16:27:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
root
 |-- month_message: long (nullable = true)
 |-- month_reaction: long (nullable = true)
 |-- month_subscription: long (nullable = true)
 |-- month_user: long (nullable = true)
 |-- zone_id: string (nullable = true)
 |-- month: date (nullable = true)
 |-- week: date (nullable = true)
 |-- week_message: long (nullable = true)
 |-- week_reaction: long (nullable = true)
 |-- week_subscription: long (nullable = true)
 |-- week_user: long (nullable = true)

+-------------+--------------+------------------+----------+-----------+----------+----------+------------+-------------+-----------------+---------+
|month_message|month_reaction|month_subscription|month_user|    zone_id|     month|      week|week_message|week_reaction|week_subscription|week_user|
+-------------+--------------+------------------+----------+-----------+----------+----------+------------+-------------

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

def create_tz_dataset(spark):

    schema = StructType([
    StructField("city", StringType(), nullable=False),
    StructField("tz_city", StringType(), nullable=False),])

    tz = [
            ('Sydney', 'Sydney'),
            ('Melbourne', 'Melbourne'),
            ('Brisbane', 'Brisbane'),
            ('Perth', 'Perth'),
            ('Adelaide', 'Adelaide'),
            ('Gold Coast', 'Brisbane'),
            ('Cranbourne', 'Melbourne'),
            ('Canberra', 'Canberra'),
            ('Newcastle', 'Sydney'),
            ('Wollongong', 'Sydney'),
            ('Geelong', 'Melbourne'),
            ('Hobart', 'Hobart'),
            ('Townsville', 'Brisbane'),
            ('Ipswich', 'Brisbane'),
            ('Cairns', 'Brisbane'),
            ('Toowoomba', 'Brisbane'),
            ('Darwin', 'Darwin'),
            ('Ballarat', 'Melbourne'),
            ('Bendigo', 'Melbourne'),
            ('Launceston', 'Hobart'),
            ('Mackay', 'Brisbane'),
            ('Rockhampton', 'Brisbane'),
            ('Maitland', 'Sydney' ),
            ('Bunbury', 'Perth')
    ]

    tz_df = spark.createDataFrame(data=tz, schema=schema)  

    return tz_df


def load_local_time(df, left_join_col, time_col, spark):

    tz = create_tz_dataset(spark)
    
    df_tz = df.join(tz, df[left_join_col] == tz["city"], "inner").drop(tz.city).withColumnRenamed("tz_city", "city")
    
    local_time = df_tz.withColumn("ts_timestamp",F.col(f"{time_col}").cast("Timestamp"))\
    .withColumn("timezone",F.concat(F.lit("Australia/"),F.col("city")))\
    .withColumn("local_time",F.from_utc_timestamp(F.col("ts_timestamp"),F.col("timezone")))
    
    return local_time


def load_recommendations(input_events_path, spark):
    
    geo_events = spark.read \
     .parquet(f"{input_events_path}")
    
    subscription_events=geo_events\
            .filter("event_type='subscription'")\
            .select("user_id", "event.subscription_channel")\
            .distinct()
    
    messages = geo_events\
            .where("event_type='message' and event.message_to is not null")\
            .select(F.col("user_id").alias("message_from"), F.col("event.message_to").alias("message_to"))\
            .distinct()

    contacts = messages\
            .union(messages.select("message_to", "message_from"))\
            .distinct()\
            .withColumn("is_contacted", F.lit(1))\
    
    pair_users = subscription_events.alias("p1")\
            .join(subscription_events.alias("p2"),"subscription_channel","inner")\
            .where("p1.user_id != p2.user_id")\
            .withColumn("user_left", F.when(F.col("p1.user_id")>F.col("p2.user_id"), F.col("p1.user_id"))\
                                                             .otherwise(F.col("p2.user_id")))\
            .withColumn("user_right", F.when(F.col("p1.user_id")>F.col("p2.user_id"), F.col("p2.user_id"))\
                                                             .otherwise(F.col("p1.user_id")))\
            .select("user_left", "user_right")\
            .distinct()
    
    pair_users = pair_users.join(contacts,[F.col("user_left") == F.col("message_from"),
                                           F.col("user_right") == F.col("message_to")], "leftanti")
    
    events_left = geo_events.select(F.col("user_id").alias("user_left"), 
                                    "date", 
                                    "ts",
                                    "lat",
                                    "lon",
                                    F.col("city").alias("zone_id"))
                       
    events_right = geo_events.select(F.col("user_id").alias("user_right"),
                                    "date",
                                     F.col("lat").alias("lat1"),
                                     F.col("lon").alias("lon1"),
                                     F.col("city").alias("zone2_id"))                  
    
    events_all=pair_users.join(events_left, "user_left", "left")\
                              .join(events_right, ["user_right","date"], "left")\
                              .withColumn("distance", 2*6378*F.sqrt(F.pow(F.sin((F.col("lat1")-F.col("lat"))/2),2) + F.cos("lat")*F.cos("lat1")*F.pow(F.sin((F.col("lon1")-F.col("lon"))/2),2)))\
                              .filter(F.col("distance")<1)\
                              .select("user_left","user_right",\
                                      F.current_timestamp().alias("processed_dttm"),\
                                      "zone_id")\
                              .distinct()        
    
    recommendations = load_local_time(events_all, "zone_id", "processed_dttm", spark)\
                            .select("user_left", "user_right", "processed_dttm", "zone_id", "local_time")
    
    return recommendations

def load_recommendations_dm(input_events_path, output_mart_path, spark):

    recommendations = load_recommendations(input_events_path, spark)

    recommendations.write.mode("overwrite").parquet(f"{output_mart_path}")  
    

In [20]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Project7") \
        .getOrCreate()

i_paths=['/user/nickperegr/prj7/prod/data/events/geo_events_cities/']
m_paths=['/user/nickperegr/prj7/prod/data/analytics/recommendations']

s = zip(i_paths, m_paths)
for i_path, m_path in s:
    load_recommendations_dm(i_path, m_path, spark)


In [21]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession \
        .builder \
        .master("local")\
        .config("spark.driver.allowMultipleContexts", "true")\
        .appName("Project 7") \
        .getOrCreate()

events = spark.read.parquet("/user/nickperegr/prj7/prod/data/analytics/recommendations/")

events.printSchema()

#events.where("home_city is not null").show(20, False)
events.show(20)

22/12/02 19:57:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
root
 |-- user_left: string (nullable = true)
 |-- user_right: string (nullable = true)
 |-- processed_dttm: timestamp (nullable = true)
 |-- zone_id: string (nullable = true)
 |-- local_time: timestamp (nullable = true)

+---------+----------+--------------------+-----------+--------------------+
|user_left|user_right|      processed_dttm|    zone_id|          local_time|
+---------+----------+--------------------+-----------+--------------------+
|    94546|     22403|2022-12-02 19:30:...| Cranbourne|2022-12-03 06:30:...|
|    79336|    163833|2022-12-02 19:30:...|  Newcastle|2022-12-03 06:30:...|
|     8093|     77752|2022-12-02 19:30:...|  Newcastle|2022-12-03 06:30:...|
|    91579|    108236|2022-12-02 19:30:...|  Newcastle|2022-12-03 06:30:...|
|    94153|    146337|2022-12-02 19:30:...|  Newcastle|2022-12-03 06:30:...|
|     1692|    118515|2022-12-02 19:30:...

In [54]:
import datetime
from datetime import datetime
from datetime import timedelta

def get_date(date, days_count):

    end_date = datetime.strptime(date, '%Y-%m-%d')
    start_date = end_date - timedelta(days=days_count)
    
    print(datetime.strftime(start_date, '%Y-%m-%d'))
    print(datetime.strftime(end_date, '%Y-%m-%d')) 

In [55]:
get_date("2022-05-01", 0)

2022-05-01
2022-05-01
